In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = "AI"

## Install libraries

In [66]:
!pip install -q youtube-transcript-api langchain-community langchain_google_genai \
faiss-cpu tiktoken python-dotenv requests==2.32.4

In [67]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

## Step 1a - Indexing (Document Ingestion)

In [68]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import TextFormatter
from youtube_transcript_api._errors import TranscriptsDisabled, NoTranscriptFound

video_id = "7ARBJQn6QkM"  # Replace with your video ID

try:
    # Fetch the transcript with language preference
    transcript_list = YouTubeTranscriptApi().fetch(video_id, languages=["en"]).to_raw_data()

    # Optionally convert to plain text using the formatter
    transcript = " ".join(item["text"] for item in transcript_list)



except TranscriptsDisabled:
        print("❌ Transcripts are disabled for this video.")
except NoTranscriptFound:
        print("❌ No transcript found in the requested languages.")
except Exception as e:
        print(f"❌ Unexpected error: {e}")


In [69]:
transcript_list

[{'text': "At some point, you have to believe something.\xa0\nWe've reinvented computing as we know it. What",
  'start': 0.08,
  'duration': 3.68},
 {'text': 'is the vision for what you see coming next? We\xa0\nasked ourselves, if it can do this, how far can',
  'start': 3.76,
  'duration': 4.72},
 {'text': 'it go? How do we get from the robots that\xa0\nwe have now to the future world that you',
  'start': 8.48,
  'duration': 4.64},
 {'text': 'see? Cleo, everything that moves will be\xa0\nrobotic someday and it will be soon. We',
  'start': 13.12,
  'duration': 4.08},
 {'text': "invested tens of billions of dollars before\xa0\nit really happened. No that's very good, you",
  'start': 17.2,
  'duration': 5.04},
 {'text': 'did some research! But the big breakthrough\xa0\nI would say is when we...',
  'start': 22.24,
  'duration': 5.994},
 {'text': "That's Jensen Huang, and whether you know it or not\nhis decisions are\xa0shaping your future. He's the CEO of",
  'start': 28.234,
  'dura

## Step 1b - Indexing (Text Splitting)

In [70]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

In [71]:
len(chunks)

71

In [72]:
chunks[60]

Document(metadata={}, page_content="use it for gaming, of course people today use it for design and creative arts and it does amazing\xa0\nAI. The real breakthrough here and this is this is truly an amazing thing, GeForce\xa0\nenabled AI and it enabled Geoff Hinton, Ilya Sutskever, Alex Krizhevsky to be able to train AlexNet. We\ndiscovered AI and we advanced AI then AI came back to GeForce to help computer graphics. And so here's\xa0\nthe amazing thing: Out of 8 million pixels or so in a 4K display we are computing, we're processing\xa0\nonly 500,000 of them. The rest of them we use AI to predict. The AI guessed it and yet the image is\xa0\nperfect. We inform it by the 500,000 pixels that we computed and we ray traced every single one and it's\xa0\nall beautiful. It's perfect. And then we tell the AI, if these are the 500,000 perfect pixels in this\xa0\nscreen, what are the other 8 million? And it goes it fills in the rest of the screen and it's perfect.")

## Step 1c & 1d - Indexing (Embedding Generation and Storing in Vector Store)

In [73]:
embeddings =GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
vector_store = FAISS.from_documents(chunks, embeddings)

In [74]:
vector_store.index_to_docstore_id

{0: 'aade6cb0-6ec3-47e3-ae6a-cb510073def8',
 1: '746e79ba-9637-4d38-8c0d-fffe30fea41d',
 2: '9970c8a3-4918-4305-a704-b505aabf1d8e',
 3: '2b13415f-10da-41d7-b38f-0a103bda5d42',
 4: '91fa045f-94b2-44bc-b3ec-fc2d9a3cd32f',
 5: '68387f91-6b73-4880-a6fd-c13731885c35',
 6: '4f13e760-ef86-48b1-8b67-6faf03b41e9d',
 7: '384aee32-b1ff-433e-99f8-dd52bdbf9dc5',
 8: 'a096961a-89ae-4905-941a-4055796e6d2d',
 9: '75b891b6-8261-41b4-8a8e-aac19c95e266',
 10: '0df35387-7609-4245-b52b-5545272b2190',
 11: '0dd3c98d-c744-484e-83f2-a0c51873bffd',
 12: '7593e47a-04c0-42d7-b64c-402064d44260',
 13: '990d8cc3-8f3b-4a71-857e-86ff613491b2',
 14: '4b120b18-4fbd-4359-83f2-e633bbfda6f9',
 15: 'ede96d70-8e6d-44aa-9f7f-c1c33a4136a8',
 16: '75d6b054-84a4-47ff-85c1-7a41770d17dc',
 17: 'a13c2e78-6ec2-4adc-823c-69016f1ad58d',
 18: 'c63e2442-2dc2-433d-8c78-4735ab0821f4',
 19: '44fbed6b-54df-432b-a758-5d05fa2dee09',
 20: 'c2172554-4610-4f52-9026-574ef4e21386',
 21: '11d8fbbb-6679-41d6-a859-0a6cbb630f31',
 22: '2012ea7a-7bab-

In [75]:
vector_store.get_by_ids(['305e3c23-14f0-4180-98c7-b720d5b3c799'])

[]

## Step 2 - Retrieval

In [76]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [77]:
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7865717e3200>, search_kwargs={'k': 4})

In [78]:
retriever.invoke('What is CUDA?')

[Document(id='0df35387-7609-4245-b52b-5545272b2190', metadata={}, page_content="oh my god it's revolutionizing all of these other industries as well, it's beginning to change\xa0\nhow we see what's possible with computers and my understanding is that in the early 2000s you\xa0\nsee this and you realize that actually doing that is a little bit difficult because what that\xa0\nresearcher had to do is he had to sort of trick the GPUs into thinking that his problem was a\xa0\ngraphics problem. That's exactly right, no that's very good, you did some research. So you create\xa0\na way to make that a lot easier. That's right Specifically it's a platform called CUDA which\xa0\nlets programmers tell the GPU what to do using programming languages that they already know like\xa0\nC and that's a big deal because it gives way more people easier access to all of this computing\xa0\npower. Could you explain what the vision was that led you to create CUDA? Partly researchers\xa0\ndiscovering it, partl

## Step 3 - Augmentation

In [79]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.2)

In [80]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [81]:
question          = "is the topic of NVIDIA DGX discussed in this video? if yes then what was discussed"
retrieved_docs    = retriever.invoke(question)

In [82]:
retrieved_docs

[Document(id='c63e2442-2dc2-433d-8c78-4735ab0821f4', metadata={}, page_content="the type of problems we could solve with machine learning is so vast that it has the potential of\xa0\nreshaping the computer industry altogether, which prompted us to re-engineer the entire\xa0\ncomputing stack which is where DGX came from and this little baby DGX sitting here, all of\xa0\nthis came from from that observation that we ought to reinvent the entire computing stack layer by\xa0\nlayer by layer. You know computers, after 65 years since IBM System 360 introduced modern general\xa0\npurpose computing, we've reinvented computing as we know it. To think about this as a whole story, so\xa0\nparallel processing reinvents modern gaming and revolutionizes an entire industry then that way\xa0\nof computing that parallel processing begins to be used across different industries. You invest\xa0\nin that by building CUDA and then CUDA and the use of GPUs allows for a a step change in neural\xa0\nnetworks an

In [83]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"the type of problems we could solve with machine learning is so vast that it has the potential of\xa0\nreshaping the computer industry altogether, which prompted us to re-engineer the entire\xa0\ncomputing stack which is where DGX came from and this little baby DGX sitting here, all of\xa0\nthis came from from that observation that we ought to reinvent the entire computing stack layer by\xa0\nlayer by layer. You know computers, after 65 years since IBM System 360 introduced modern general\xa0\npurpose computing, we've reinvented computing as we know it. To think about this as a whole story, so\xa0\nparallel processing reinvents modern gaming and revolutionizes an entire industry then that way\xa0\nof computing that parallel processing begins to be used across different industries. You invest\xa0\nin that by building CUDA and then CUDA and the use of GPUs allows for a a step change in neural\xa0\nnetworks and machine learning and begins a sort of revolution that we're now seeing only\n

In [84]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [85]:
final_prompt

StringPromptValue(text="\n      You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      the type of problems we could solve with machine learning is so vast that it has the potential of\xa0\nreshaping the computer industry altogether, which prompted us to re-engineer the entire\xa0\ncomputing stack which is where DGX came from and this little baby DGX sitting here, all of\xa0\nthis came from from that observation that we ought to reinvent the entire computing stack layer by\xa0\nlayer by layer. You know computers, after 65 years since IBM System 360 introduced modern general\xa0\npurpose computing, we've reinvented computing as we know it. To think about this as a whole story, so\xa0\nparallel processing reinvents modern gaming and revolutionizes an entire industry then that way\xa0\nof computing that parallel processing begins to be used across different industries. You invest\xa0\ni

## Step 4 - Generation

In [86]:
answer = llm.invoke(final_prompt)
print(answer.content)

Yes, the topic of NVIDIA DGX is discussed in the transcript.

Here's what was discussed:
*   DGX originated from the observation that the entire computing stack needed to be reinvented due to the vast potential of machine learning.
*   The very first version, DGX 1, was delivered to OpenAI in 2016 and cost $250,000.
*   A "little baby DGX" or "mini mini version" was created because AI has expanded beyond researchers to every engineer, student, and AI scientist.
*   These smaller versions will cost $3,000, making them accessible for schools and students to have their own AI supercomputer.
*   The DGX 1 required 10,000 times more energy than the current version, which has six times more performance.
*   Since 2016, the energy efficiency of computing has increased by 10,000 times.


## Building a Chain

In [87]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [88]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [89]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [90]:
parallel_chain.invoke('What’s Jensen’s next bet?')

{'context': 'At some point, you have to believe something.\xa0\nWe\'ve reinvented computing as we know it. What is the vision for what you see coming next? We\xa0\nasked ourselves, if it can do this, how far can it go? How do we get from the robots that\xa0\nwe have now to the future world that you see? Cleo, everything that moves will be\xa0\nrobotic someday and it will be soon. We invested tens of billions of dollars before\xa0\nit really happened. No that\'s very good, you did some research! But the big breakthrough\xa0\nI would say is when we... That\'s Jensen Huang, and whether you know it or not\nhis decisions are\xa0shaping your future. He\'s the CEO of NVIDIA, the company that skyrocketed over the past few\nyears\xa0to become one of the most valuable companies in the world because they led a fundamental shift\xa0\nin how computers work unleashing this current explosion of what\'s possible with technology.\xa0\n"NVIDIA\'s done it again!" We found ourselves being one of the most 

In [91]:
parser = StrOutputParser()

In [92]:
main_chain = parallel_chain | prompt | llm | parser

In [93]:
main_chain.invoke('Can you summarize the video')

'The transcript discusses several topics:\n\n*   **AI Tutors and Super AIs**: It strongly encourages everyone to get an AI tutor, highlighting its ability to teach, help with programming, writing, analysis, thinking, and reasoning, ultimately empowering individuals to become "super humans" through "super AIs."\n*   **GeForce RTX 50 Series Graphics Card**: This is described as a new graphics card, essentially a supercomputer for PCs, used for gaming, design, creative arts, and excelling in AI capabilities.\n*   **Core Beliefs and First Principles**: The importance of reasoning from fundamental principles (like physics or deep industry/science understanding) and maintaining core beliefs even when immediate evidence of success is lacking.\n*   **Scalability and Learning Capabilities of AI/Deep Learning**: The belief that AI can scale without physical or architectural limits and its demonstrated ability to learn from almost any data modality (images, sound, text) and translate between them